In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import seaborn as sns
from sklearn.model_selection import train_test_split
from ydata_profiling import ProfileReport
import numpy as np
from sklearn.model_selection import GridSearchCV

In [3]:
df = pd.read_csv('voiture.csv')


In [31]:
df['marque'] = df['marque'].replace('maxda', 'mazda')
df['marque'] = df['marque'].replace('Nissan', 'nissan')
df['marque'] = df['marque'].replace('Nissan', 'nissan')
df['marque'] = df['marque'].replace('porcshce', 'porsche')
df['marque'] = df['marque'].replace('toyouta', 'toyota')
df['marque'] = df['marque'].replace('vokswagen', 'volkswagen')
df['marque'] = df['marque'].replace('vw', 'volkswagen')
df['marque'] = df['marque'].replace('alfa-romero', 'alpha-romeo')

In [33]:
df.to_csv("voiture_final.csv")

In [32]:
df.at[110, 'modele'] = '504 turbo break'
df.at[116, 'modele'] = '504 turbo'
df.at[113, 'modele'] = '504 essence break'
df.at[115, 'modele'] = '504 gas 97ch'
df.at[111, 'modele'] = '504 gas 95ch'

In [17]:
aa=df.loc[df['modele']=='504']

In [18]:
df = df.rename(columns={'symboling': 'etat_de_route',
                        'CarName': 'marque_voiture',
                        'fueltype': 'carburant',
                        'aspiration': 'turbo',
                        'doornumber': 'nombre_portes',
                        'carbody': 'type_vehicule',
                        'drivewheel': 'roues_motrices', 
                        'enginelocation': 'emplacement_moteur',
                        'wheelbase': 'empattement', 
                        'carlength': 'longueur_voiture', 
                        'carwidth': 'largeur_voiture', 
                        'carheight': 'hauteur_voiture', 
                        'curbweight': 'poids_vehicule', 
                        'enginetype': 'type_moteur', 
                        'cylindernumber': 'nombre_cylindres', 
                        'enginesize': 'taille_moteur', 
                        'fuelsystem': 'systeme_carburant', 
                        'boreratio': 'taux_alésage', 'stroke': 
                        'course', 'compressionratio': 'taux_compression', 
                        'horsepower': 'chevaux', 'peakrpm': 'tour_moteur', 
                        'citympg': 'consommation_ville', 
                        'highwaympg': 'consommation_autoroute', 
                        'price': 'prix'})

### colonne marque_voiture : séparer les marques et les modeles.

In [19]:

df[['marque', 'modele']] = df['marque_voiture'].str.split(' ', 1, expand=True)
df = df.drop('marque_voiture', axis=1)


/tmp/ipykernel_9987/2267248324.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['marque', 'modele']] = df['marque_voiture'].str.split(' ', 1, expand=True)


 ### colonne turbo : Remplacer std par standard.



In [20]:
df['turbo'] = df['turbo'].replace('std', 'standard')
print(df['turbo'].unique())

['standard' 'turbo']


### colonne nombre_portes : nombre de roues a convertir en chiffres.

In [21]:
df['nombre_portes'] = df['nombre_portes'].replace({'two': '2', 'four': '4'}).astype(int)
print(df['nombre_portes'].unique())

[2 4]


### colonne 7 : Traduire les valeurs en français.

In [22]:
df['type_vehicule'] = df['type_vehicule'].replace({'convertible': 'decapotable', 'hatchback': 'hayon', 'sedan': 'berline', 'wagon': 'break', 'hardtop': 'coupe'})

print(df['type_vehicule'].unique())

['decapotable' 'hayon' 'berline' 'break' 'coupe']



    Convertible : décapotable
    Hatchback : hayon (ou voiture à hayon)
    Sedan : berline
    Wagon : break
    Hardtop : coupé sans montants de toit apparents (c'est-à-dire avec une ligne de toit continue de l'avant à l'arrière)

### colonne roues_motrices : traduire en français.

In [23]:
df['roues_motrices'] = df['roues_motrices'].replace({'rwd': 'arriere', 'fwd': 'avant', '4wd': '4motrice',})

print(df['roues_motrices'].unique())

['arriere' 'avant' '4motrice']


In [1]:
df.corr()

NameError: name 'df' is not defined

### colonne emplacement_moteur : Traduire en français.

In [24]:
df['emplacement_moteur'] = df['emplacement_moteur'].replace({'front': 'devant', 'rear': 'derrier',})

print(df['emplacement_moteur'].unique())

['devant' 'derrier']


### colonnes empattement, longueur_voiture, largeur_voiture, hauteur_voiture, poids_vehicule : convertir en metrique

 - colonne empattement : pouces en metres
 - colonne longueur_voiture : pouces en metres
 - colonne largeur_voiture : pouces en metres
 - colonne hauteur_voiture : pouces en metres
 - colonne poids_vehicule : livres tonnes

In [25]:
df['empattement'] = df['empattement'] / 39.37
df['longueur_voiture'] = df['longueur_voiture'] / 39.37
df['largeur_voiture'] = df['largeur_voiture'] / 39.37
df['hauteur_voiture'] = df['hauteur_voiture'] / 39.37
df['poids_vehicule'] = df['poids_vehicule'] / 2000

In [26]:
df['empattement'] = df['empattement'].round(2)
df['longueur_voiture'] = df['longueur_voiture'].round(2)
df['largeur_voiture'] = df['largeur_voiture'].round(2)
df['hauteur_voiture'] = df['hauteur_voiture'].round(2)
df['poids_vehicule'] = df['poids_vehicule'].round(2)


### colonne type_moteur : Traduire en français.

 - 'dohc', : double ACT
 - 'ohcv', : soupapes en tête à manchon
 - 'ohc',  : ACT
 - 'l',    : ligne des cylindres
 - 'rotor',: moteur rotatif
 - 'ohcf', : soupapes en tête à flux croisés
 - 'dohcv' : double ACT à soupapes en V

In [27]:
df['type_moteur'] = df['type_moteur'].replace({'dohc': 'double ACT',
                                                'ohcv': 'soupapes en tête à manchon',
                                                'ohc': 'ACT',
                                                'l': 'ligne des cylindres',
                                                'rotor': 'moteur rotatif',
                                                'ohcf': 'soupapes en tête à flux croisés',
                                                'dohcv': 'double ACT à soupapes en V',})

df['type_moteur'].unique()

array(['double ACT', 'soupapes en tête à manchon', 'ACT',
       'ligne des cylindres', 'moteur rotatif',
       'soupapes en tête à flux croisés', 'double ACT à soupapes en V'],
      dtype=object)

In [28]:
df["nombre_cylindres"].replace({"two": 2, "three":3, "four":4, "five":5, "six":6, "eight":8, "twelve":12}, inplace=True)
df.loc[:, 'taille_moteur'] = round(df.loc[:, 'taille_moteur'] * 16.387)
df["systeme_carburant"].replace({"mpfi": "injection multipoint multipoint port unique", "2bbl": "carburateur 2 corps", "mfi":"injection carburant multipoint", "1bbl":"carburateur 1 corps", "spfi": "injection monopoint", "4bbl":"carburateur 4 corps", "idi":"injection indirecte", "spdi":"injection D monopoint"}, inplace=True)

df['consommation_ville'] = 235.214583/df['consommation_ville']
df['consommation_autoroute'] = 235.214583/df['consommation_autoroute']

/tmp/ipykernel_9987/3725372285.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'taille_moteur'] = round(df.loc[:, 'taille_moteur'] * 16.387)


In [29]:
df = df.rename(columns={'taille_moteur': 'moteur_cc3'})

In [30]:
y = df['prix']# ou df.drop("price", axis=1) pr dire qu'on suppr 1 colonne)
#X = df[["longueur_voiture","largeur_voiture","empattement","nombre_cylindres","moteur_cc3","taux_alésage","chevaux","consommation_ville","consommation_autoroute"]]
X = df[["longueur_voiture","largeur_voiture","empattement","nombre_cylindres","moteur_cc3","chevaux","consommation_ville","consommation_autoroute"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [ ]:
df.corr()

/tmp/ipykernel_11035/1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,car_ID,etat_de_route,nombre_portes,empattement,longueur_voiture,largeur_voiture,hauteur_voiture,poids_vehicule,nombre_cylindres,moteur_cc3,taux_alésage,course,taux_compression,chevaux,tour_moteur,consommation_ville,consommation_autoroute,prix
car_ID,1.000000,-0.153056,0.192217,0.132812,0.176858,0.062782,0.258044,0.078967,-0.092530,-0.030389,0.255316,-0.150693,0.152009,-0.010567,-0.200893,-0.086523,-0.067933,-0.104148
etat_de_route,-0.153056,1.000000,-0.661474,-0.529742,-0.355320,-0.245507,-0.552426,-0.227850,-0.113103,-0.104538,-0.132650,-0.002115,-0.178352,0.073560,0.274903,0.064587,-0.029634,-0.078380
nombre_portes,0.192217,-0.661474,1.000000,0.445709,0.395728,0.208386,0.562422,0.196744,-0.016530,0.018844,0.121864,-0.019072,0.177940,-0.130456,-0.249012,-0.035329,0.019535,0.029780
empattement,0.132812,-0.529742,0.445709,1.000000,0.875576,0.805104,0.594631,0.776704,0.336982,0.567747,0.499073,0.153479,0.247272,0.351404,-0.364372,0.472851,0.577252,0.575295
longueur_voiture,0.176858,-0.355320,0.395728,0.875576,1.000000,0.843830,0.494111,0.877831,0.430784,0.683317,0.622034,0.112482,0.156519,0.550760,-0.292961,0.658438,0.711687,0.681352
largeur_voiture,0.062782,-0.245507,0.208386,0.805104,0.843830,1.000000,0.293089,0.868911,0.539637,0.732134,0.576379,0.164671,0.185744,0.632568,-0.230316,0.675431,0.723801,0.756895
hauteur_voiture,0.258044,-0.552426,0.562422,0.594631,0.494111,0.293089,1.000000,0.300932,-0.017839,0.068292,0.181325,-0.059491,0.260261,-0.110295,-0.322088,-0.002428,0.089538,0.120855
poids_vehicule,0.078967,-0.227850,0.196744,0.776704,0.877831,0.868911,0.300932,1.000000,0.609702,0.850423,0.664808,0.153083,0.151115,0.748894,-0.273513,0.789433,0.834638,0.833389
nombre_cylindres,-0.092530,-0.113103,-0.016530,0.336982,0.430784,0.539637,-0.017839,0.609702,1.000000,0.846046,0.236627,-0.000192,-0.020803,0.691805,-0.126475,0.561112,0.588558,0.718537
moteur_cc3,-0.030389,-0.104538,0.018844,0.567747,0.683317,0.732134,0.068292,0.850423,0.846046,1.000000,0.594402,0.194800,0.027726,0.808972,-0.248693,0.744112,0.776337,0.873894


In [ ]:
model = LinearRegression()
trained_model = model.fit(X_train,y_train)
trained_model.score(X_test,y_test)
# model.predict(X)


0.6926004218209902

# nettoyage de données

In [25]:
df = df.dropna(subset=['modele'])

In [ ]:
profile=ProfileReport(df, title="vignerons")
profile.to_file('ici.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
y = df['prix']
X = df[["longueur_voiture","largeur_voiture","empattement","nombre_cylindres","moteur_cc3","chevaux","consommation_ville","consommation_autoroute","tour_moteur"]]

# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

# Créer un objet de modèle Lasso et l'entraîner sur les données d'entraînement
lasso_model = Lasso(alpha=0.1)
trained_model = lasso_model.fit(X_train, y_train)

# Évaluer les performances du modèle sur les données de test
score = trained_model.score(X_test, y_test)
print("Score du modèle Lasso : ", score)

# Prédire les valeurs de prix à partir des données X
predictions = trained_model.predict(X)


Score du modèle Lasso :  0.6989412407181168


In [ ]:

# Sélectionner les colonnes à utiliser dans le modèle
y = df['prix']
X = df[["longueur_voiture","largeur_voiture","empattement","nombre_cylindres","moteur_cc3","chevaux","consommation_ville","consommation_autoroute"]]

# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

# Créer un objet de modèle Ridge et l'entraîner sur les données d'entraînement
ridge_model = Ridge(alpha=0.1)
trained_model = ridge_model.fit(X_train, y_train)

# Évaluer les performances du modèle sur les données de test
score = trained_model.score(X_test, y_test)
print("Score du modèle Ridge : ", score)

# Prédire les valeurs de prix à partir des données X
predictions = trained_model.predict(X)

Score du modèle Ridge :  0.7013881649538309


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Sélectionner les colonnes à utiliser dans le modèle
y = df['prix']
X_cat = df[["marque", "modele"]]
X_num= df[["longueur_voiture","largeur_voiture","empattement","nombre_cylindres","moteur_cc3","chevaux","consommation_ville","consommation_autoroute"]]

# Appliquer OneHotEncoder sur les variables catégorielles
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_cat_encoded = encoder.fit_transform(X_cat)

# Concaténer les variables numériques et binaires pour obtenir les données finales
import numpy as np
X = np.concatenate((X_num, X_cat_encoded), axis=1)

# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

# Créer un objet de modèle Ridge et l'entraîner sur les données d'entraînement
ridge_model = Ridge(alpha=0.1)
trained_model = ridge_model.fit(X_train, y_train)

# Évaluer les performances du modèle sur les données de test
score = trained_model.score(X_test, y_test)
print("Score du modèle Ridge avec OneHotEncoder : ", score)

# Prédire les valeurs de prix à partir des données X
predictions = trained_model.predict(X)

/home/apprenant/miniconda3/envs/sklearn-env/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


AttributeError: 'numpy.ndarray' object has no attribute 'fit_transform'

In [ ]:
df.columns

Index(['car_ID', 'etat_de_route', 'carburant', 'turbo', 'nombre_portes',
       'type_vehicule', 'roues_motrices', 'emplacement_moteur', 'empattement',
       'longueur_voiture', 'largeur_voiture', 'hauteur_voiture',
       'poids_vehicule', 'type_moteur', 'nombre_cylindres', 'moteur_cc3',
       'systeme_carburant', 'taux_alésage', 'course', 'taux_compression',
       'chevaux', 'tour_moteur', 'consommation_ville',
       'consommation_autoroute', 'prix', 'marque', 'modele'],
      dtype='object')

In [ ]:
y = df['prix']
X_cat = df[["marque", "modele"]]
# X_num = df[["longueur_voiture","largeur_voiture","empattement","nombre_cylindres","moteur_cc3","chevaux","consommation_ville","consommation_autoroute"]]
X_num=df[['etat_de_route', 'carburant', 'turbo', 'nombre_portes',
       'type_vehicule', 'roues_motrices', 'emplacement_moteur', 'empattement',
       'longueur_voiture', 'largeur_voiture', 'hauteur_voiture',
       'poids_vehicule', 'type_moteur', 'nombre_cylindres', 'moteur_cc3',
       'systeme_carburant', 'taux_alésage', 'course', 'taux_compression',
       'chevaux', 'tour_moteur', 'consommation_ville',
       'consommation_autoroute']]
# Appliquer OneHotEncoder sur les variables catégorielles
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_cat_encoded = encoder.fit_transform(X_cat)

# Concaténer les variables numériques et binaires pour obtenir les données finales
import numpy as np
X = np.concatenate((X_num, X_cat_encoded), axis=1)

# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

# Définir la grille de valeurs d'hyperparamètres à tester
param_grid = {'alpha': [0.1, 1, 10, 100]}

# Créer un objet de modèle Ridge
ridge_model = Ridge()

# Créer un objet GridSearchCV pour trouver les meilleures valeurs d'hyperparamètres
grid_search = GridSearchCV(estimator=ridge_model, param_grid=param_grid, cv=5)

# Entraîner le modèle sur les données d'entraînement en utilisant la recherche de grille
trained_model = grid_search.fit(X_train, y_train)

# Afficher les meilleures valeurs d'hyperparamètres trouvées
print("Meilleures valeurs d'hyperparamètres : ", trained_model.best_params_)

# Évaluer les performances du modèle sur les données de test
score = trained_model.score(X_test, y_test)
print("Score du modèle Ridge avec OneHotEncoder : ", score)

# Prédire les valeurs de prix à partir des données X
predictions = trained_model.predict(X)

/home/apprenant/miniconda3/envs/sklearn-env/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ValueError: 
All the 20 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/home/apprenant/miniconda3/envs/sklearn-env/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/apprenant/miniconda3/envs/sklearn-env/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py", line 1126, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/apprenant/miniconda3/envs/sklearn-env/lib/python3.11/site-packages/sklearn/base.py", line 584, in _validate_data
    X, y = check_X_y(X, y, **check_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/apprenant/miniconda3/env

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer  
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
# Sélectionner la variable cible et les variables explicatives


Y = df['prix']
X=df[['etat_de_route', 'nombre_portes',
        'empattement',
       'longueur_voiture', 'largeur_voiture', 'hauteur_voiture',
       'poids_vehicule',  'nombre_cylindres', 'moteur_cc3',
        'taux_alésage', 'course', 'taux_compression',
       'chevaux', 'tour_moteur', 'consommation_ville',
       'consommation_autoroute',"marque", "modele", 'carburant', 'turbo','type_vehicule', 'roues_motrices', 'emplacement_moteur','type_moteur','systeme_carburant']]


X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=42)



X_num = df[['etat_de_route', 'nombre_portes','empattement', 'longueur_voiture', 'largeur_voiture', 'hauteur_voiture','poids_vehicule',  'nombre_cylindres', 'moteur_cc3',
'taux_alésage', 'course', 'taux_compression','chevaux', 'tour_moteur', 'consommation_ville','consommation_autoroute']]

# Sélectionner les colonnes catégorielles
X_cat = df[["marque", "modele", 'carburant', 'turbo','type_vehicule', 'roues_motrices', 'emplacement_moteur','type_moteur','systeme_carburant']]

# Encoder les variables catégorielles en variables binaires
ttt_cat = OneHotEncoder(handle_unknown='ignore', sparse=False)#.fit_transform(X_cat)
ttt_num=MinMaxScaler()#.fit_transform(X_num)



preprocessor = ColumnTransformer(                   
    transformers=[ 
        ('num', ttt_num, X_num.columns),
        ('cat', ttt_cat, X_cat.columns)        
    ])

ridge = Lasso()

pipe = Pipeline([
     ('prep', preprocessor),     
     ('knn', ridge)              
])

# trained_pipe = pipe.fit(X_train,y_train)

# # prediction sur X_test
# trained_pipe.predict(X_test)

# # scoring sur X_test
# trained_pipe.score(X_test,y_test)














liste = np.arange(1, 12, 0.1).tolist()
#quelle_facon = ['uniform','distance']       #   uniform: attribue un poids égal à tous les voisins     # distance: pondére chak voisin en fonction de sa distance 

param_grid = dict(alpha=liste)

knn = Lasso() 
grid = GridSearchCV(knn, param_grid, cv=2, scoring='r2') 

grid.fit (X_train, y_train) 

grid.best_score_
grid.best_params_


# Concaténer les variables numériques et binaires pour obtenir les données finales
# X = np.concatenate((X_num, X_cat_encode), axis=1)

# # Séparer les données en ensembles d'entraînement et de test


# # Définir la grille de valeurs d'hyperparamètres à tester
# param_grid = {'alpha': [3,4,2,4,5,6,7,8,9, 10, 100]}

# # Créer un objet de modèle Ridge
# ridge_model = Ridge()

# # Créer un objet GridSearchCV pour trouver les meilleures valeurs d'hyperparamètres
# grid_search = GridSearchCV(estimator=ridge_model, param_grid=param_grid, cv=5)

# # Entraîner le modèle sur les données d'entraînement en utilisant la recherche de grille
# trained_model = grid_search.fit(X_train, y_train)

# # Afficher les meilleures valeurs d'hyperparamètres trouvées
# print("Meilleures valeurs d'hyperparamètres : ", trained_model.best_params_)
# score = trained_model.score(X_test, y_test)
# print("Score du modèle Ridge avec OneHotEncoder : ", score)


ValueError: 
All the 220 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
110 fits failed with the following error:
Traceback (most recent call last):
  File "/home/apprenant/miniconda3/envs/sklearn-env/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/apprenant/miniconda3/envs/sklearn-env/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py", line 908, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/apprenant/miniconda3/envs/sklearn-env/lib/python3.11/site-packages/sklearn/base.py", line 584, in _validate_data
    X, y = check_X_y(X, y, **check_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/apprenant/

In [ ]:
numeric_transformer = Pipeline([       
        ('imputer', SimpleImputer(strategy='median')),
        ('min_max', MinMaxScaler()),  
        ])

categorial_var = [ "zipcode","grade","waterfront"]

remaining_var=["date"]

from sklearn.preprocessing import OneHotEncoder
categorical_transformer = OneHotEncoder(sparse=True)   

from sklearn.compose import ColumnTransformer       
# #preprocessor = ColumnTransformer(                   
#     transformers=[ 
#         ('num', numeric_transformer, numeric_var),
#         ('cat', categorical_transformer, categorial_var),
#         ('remainder':‘passthrough’)
#     ])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_var),
        ('cat', categorical_transformer, categorial_var),
        # ('remainder', 'passthrough', remaining_var)
    ])

from sklearn.neighbors import KNeighborsRegressor
knn_4 = KNeighborsRegressor(n_neighbors=7, weights="distance")


pipe = Pipeline([
     ('prep', preprocessor),     
     ('knn', knn_4)              
])

trained_pipe = pipe.fit(X_train,y_train)

# prediction sur X_test
trained_pipe.predict(X_test)

# scoring sur X_test
trained_pipe.score(X_test,y_test)


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

import numpy as np
X= np.array ([[70],[80],[100], [120],[140]])
X_bis=np.array ([[79],[180]])
scaler=StandardScaler()
A=scaler.fit_transform(X)
scaler.transform(X_bis)


AttributeError: 'numpy.ndarray' object has no attribute 'inverse_transform'

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer  
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline

# Sélectionner la variable cible et les variables explicatives
Y = df['prix']
X = df[['etat_de_route', 'nombre_portes',        'empattement',        'longueur_voiture', 'largeur_voiture', 'hauteur_voiture',        'poids_vehicule', 'nombre_cylindres', 'moteur_cc3',        'taux_alésage', 'course', 'taux_compression',        'chevaux', 'tour_moteur', 'consommation_ville',        'consommation_autoroute', "marque", "modele", 'carburant', 'turbo',        'type_vehicule', 'roues_motrices', 'emplacement_moteur', 'type_moteur',        'systeme_carburant']]

X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=42)

X_num = X[['etat_de_route', 'nombre_portes', 'empattement', 'longueur_voiture', 'largeur_voiture', 'hauteur_voiture', 'poids_vehicule', 'nombre_cylindres', 'moteur_cc3', 'taux_alésage', 'course', 'taux_compression', 'chevaux', 'tour_moteur', 'consommation_ville', 'consommation_autoroute']]
X_cat = X[["marque", "modele", 'carburant', 'turbo', 'type_vehicule', 'roues_motrices', 'emplacement_moteur', 'type_moteur', 'systeme_carburant']]

ttt_num = Pipeline(steps=[('scaler', MinMaxScaler())])
ttt_cat = Pipeline(steps=[('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))])

preprocessor = ColumnTransformer(transformers=[
    ('num', ttt_num, X_num.columns),
    ('cat', ttt_cat, X_cat.columns)
])

ridge = Lasso()

pipe = Pipeline([
    ('prep', preprocessor),     
    ('knn', ridge)              
])
# pipe.set_params(knn__alpha=4.7)

param_grid = {'knn': [0.1, 0.5, 1.0, 2.0, 4,4.5,4.6,4.7,4.8,4.9,5.0,5.1,5.4,5.9]} # valeurs à tester pour alpha
grid_search = GridSearchCV(pipe, param_grid, cv=5) # définition de l'objet GridSearchCV
grid_search.fit(X_train, y_train) # ajustement avec GridSearchCV sur les données d'entraînement
print(grid_search.best_params_) # affichage des meilleurs paramètres
print(grid_search.best_score_)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer  
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures

from sklearn.pipeline import Pipeline

# Sélectionner la variable cible et les variables explicatives
Y = df['prix']
X = df[['etat_de_route', 'nombre_portes',        'empattement',        'longueur_voiture', 'largeur_voiture', 'hauteur_voiture',        'poids_vehicule', 'nombre_cylindres', 'moteur_cc3',        'taux_alésage', 'course', 'taux_compression',        'chevaux', 'tour_moteur', 'consommation_ville',        'consommation_autoroute', "marque", "modele", 'carburant', 'turbo',        'type_vehicule', 'roues_motrices', 'emplacement_moteur', 'type_moteur',        'systeme_carburant']]

X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=42)

X_num = X[['etat_de_route', 'nombre_portes', 'empattement', 'longueur_voiture', 'largeur_voiture', 'hauteur_voiture', 'poids_vehicule', 'nombre_cylindres', 'moteur_cc3', 'taux_alésage', 'course', 'taux_compression', 'chevaux', 'tour_moteur', 'consommation_ville', 'consommation_autoroute']]
X_cat = X[["marque", "modele", 'carburant', 'turbo', 'type_vehicule', 'roues_motrices', 'emplacement_moteur', 'type_moteur', 'systeme_carburant']]

ttt_num = Pipeline(steps=[('scaler', MinMaxScaler()), ('poly', PolynomialFeatures(degree=2))])
ttt_cat = Pipeline(steps=[('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))])

preprocessor = ColumnTransformer(transformers=[
    ('num', ttt_num, X_num.columns),
    ('cat', ttt_cat, X_cat.columns)
])

ridge = Lasso()

pipe = Pipeline([
    ('prep', preprocessor),     
    ('knn', ridge)              
])
# pipe.set_params(knn__alpha=4.7)

param_grid = {'knn__alpha': [0.1, 0.5, 1.0, 2.0, 4,4.5,4.6,4.7,4.8,4.9,5.0,5.1,5.4,5.9]} # valeurs à tester pour alpha
grid_search = GridSearchCV(pipe, param_grid, cv=5) # définition de l'objet GridSearchCV
grid_search.fit(X_train, y_train) # ajustement avec GridSearchCV sur les données d'entraînement
print(grid_search.best_params_) # affichage des meilleurs paramètres
print(grid_search.best_score_)
trained_pipe = pipe.fit(X_train, y_train)

# # prediction sur X_test
# trained_pipe.predict(X_test)

# # scoring sur X_test
# trained_pipe.score(X_test, y_test)

pipeline = pipe
#=make_pipeline(StandardScaler(),
 #                       LogisticRegression(penalty='l2', solver='lbfgs', random_state=1, max_iter=10000))
#
# Use learning curve to get training and test scores along with train sizes
#
train_sizes, train_scores, test_scores = learning_curve(estimator=pipeline, X=X_train, y=y_train,
                                                       cv=10, train_sizes=np.linspace(0.1, 1.0, 10),
                                                     n_jobs=1)
# #
# # Calculate training and test mean and std
# #
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
# #
# # Plot the learning curve
# #
plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training Accuracy')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
plt.plot(train_sizes, test_mean, color='green', marker='+', markersize=5, linestyle='--', label='Validation Accuracy')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
plt.title('Learning Curve')
plt.xlabel('Training Data Size')
plt.ylabel('Model accuracy')
plt.grid()
plt.legend(loc='lower right')
plt.savefig('learning_curve.png')
plt.show()

NameError: name 'df' is not defined

In [ ]:
from sklearn.model_selection import GridSearchCV

# Définition de la grille de paramètres pour KNeighborsRegressor
k_range = list(range(1, 12))
quelle_facon = ['uniform', 'distance']
param_grid = dict(n_neighbors=k_range, weights=quelle_facon)

# Définition des modèles
ridge = Ridge()
knn = Lasso()

# Prétraitement des données
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols)
    ])

# Pipeline pour Ridge
ridge_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('polynomial_features', poly),
    ('ridge', ridge)
])

# Pipeline pour KNeighborsRegressor
knn_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('polynomial_features', poly),
    ('knn', knn)
])

# Définition de la grille de paramètres pour Ridge
param_grid_ridge = {
    'polynomial_features__degree': [1, 2, 3],
    'ridge__alpha': [0.001, 0.01, 0.1, 1, 10, 100]
}

# Recherche de grille pour Ridge
grid_ridge = GridSearchCV(ridge_pipe, param_grid=param_grid_ridge, cv=5, scoring='r2')
grid_ridge.fit(X_train, y_train)

# Affichage du meilleur score et des meilleurs paramètres pour Ridge
print("Best Score for Ridge: ", grid_ridge.best_score_)
print("Best Parameters for Ridge: ", grid_ridge.best_params_)

# Définition de la grille de paramètres pour KNeighborsRegressor
param_grid_knn = {
    'polynomial_features__degree': [1, 2, 3],
    'knn__n_neighbors': k_range,
    'knn__weights': quelle_facon
}

# Recherche de grille pour KNeighborsRegressor
grid_knn = GridSearchCV(knn_pipe, param_grid=param_grid_knn, cv=5, scoring='r2')
grid_knn.fit(X_train, y_train)

# Affichage du meilleur score et des meilleurs paramètres pour KNeighborsRegressor
print("Best Score for KNeighborsRegressor: ", grid_knn.best_score_)
print("Best Parameters for KNeighborsRegressor: ", grid_knn.best_params_)

NameError: name 'KNeighborsRegressor' is not defined

In [29]:
dico={'etat_de_route': etat_de_route, 'nombre_portes': nombre_portes, 
                      'empattement': empattement, 'longueur_voiture': longueur, 
                      'largeur_voiture': largeur, 'hauteur_voiture': hauteur, 
                      'poids_vehicule': poids, 'nombre_cylindres': nombre_cylindres, 
                      'moteur_cc3': moteur_cc3, 'taux_alésage': taux_alésage,  
                      'taux_compression': taux_compression, 'chevaux': chevaux, 
                      'tour_moteur': tour_moteur, 'consommation_ville': consommation_ville, 
                      'consommation_autoroute': consommation_autoroute, 'marque': marque, 
                      'modele': modele, 'carburant': carburant, 'turbo': turbo, 
                      'type_vehicule': type_vehicule, 'roues_motrices': roues_motrices, 
                      'emplacement_moteur': emplacement_moteur, 'type_moteur': type_moteur, 
                      'systeme_carburant': systeme_carburant}
taille = len(dico)
print(taille)

NameError: name 'etat_de_route' is not defined

In [28]:
import pickle

def load_model():
    with open('model.pkl', 'rb') as file:
        model = pickle.load(file)
    print(f"Number of expected features: {model.n_features_in_}")
    return model

load_model()

Number of expected features: 25


/home/apprenant/miniconda3/envs/sklearn-env/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/apprenant/miniconda3/envs/sklearn-env/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/apprenant/miniconda3/envs/sklearn-env/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OneHotEncoder from version 1.2.1 when using versio

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler())]),
                                                  Index(['etat_de_route', 'nombre_portes', 'empattement', 'longueur_voiture',
       'largeur_voiture', 'hauteur_voiture', 'poids_vehicule',
       'nombre_cylindres', 'moteur_cc3', 'taux_alésage', 'course',
       'taux_compression', 'chevaux', 'tour_moteur', 'consommation_ville',
       'consommation_autoroute'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  Index(['marque', 'modele', 'carburant', 'turbo', 'type_vehicule',
       'roues_motrices', 'emplacement_moteur', 'type_moteur',
       'systeme_carburant'],
      dtype='object'))])),
                ('knn', Lasso())])